 정수 n 이 주어졌을 때 n 번째 피보나치 수열의 항을 계산하는 함수를 작성하시오.
 
 예)
 $fib(0) = 0$ 

 $fib(1) = 1$
 
 $fib(5) = 5$ 
 
 $fib(10) = 55$
 
 ## 첫 번째 해법 찾기
 
 FAST 방법의 첫 번째 단계는 가능한 첫 번째 해법을 찾는 것이다. 이런 종류의 문제에서는 피보나치 수열의 정의에 집중한다면 그 해법을 찾을 수 있다. 만약 정의가 확실치 않다면 인터뷰어에게 물어보면 된다. 하지만, 대부분의 사람들이 학교에서 다음의 개념을 배웠을 것이다.

$fib(n) = fib(n-1) + fib(n-2)$

$fib(0) = 0$

$fib(1) = 1$

위의 세 표현식만 사용한다면, 모든 피보나치 수열 문제를 풀 수 있다.

세 표현식을 자세히 살펴보면 이미 재귀적 함수를 정의했다는 것을 알 수 있다. $f(0), f(1)$ 이렇게 두 개의 기본 케이스가 있다. $fib(n) = fib(n-1) + fib(n - 2)$는 재귀적 호출에 해당한다. 따라서, 첫 번째 해법을 다음과 같이 작성할 수 있다.

In [12]:
'''
Fig 1. Naive recursive Fibonacci solution
Compute the nth Fibonacci number
We assume that n >= 0 and that int is sufficient to hold the result
'''

def fib(n):
    if n == 0:
        return 0
    if n == 1:
        return 1
    return fib(n-1) + fib(n-2)


In [11]:
fib(7)

13